In [1]:
import os
import pandas as pd
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tensorflow.data import Dataset

from pathlib import Path
import pandas as pd

print(cv2.__version__)

2025-05-06 20:26:45.507524: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-06 20:26:45.759620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746534405.843571    2144 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746534405.877153    2144 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 20:26:46.096048: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

4.11.0


In [3]:
df = pd.read_csv("../data/csv/Data_Entry_2017_v2020.csv")
print(df.head(5))

        Image Index          Finding Labels  Follow-up #  Patient ID  \
0  00000001_000.png            Cardiomegaly            0           1   
1  00000001_001.png  Cardiomegaly|Emphysema            1           1   
2  00000001_002.png   Cardiomegaly|Effusion            2           1   
3  00000002_000.png              No Finding            0           2   
4  00000003_001.png                  Hernia            0           3   

   Patient Age Patient Sex View Position  OriginalImage[Width  Height]  \
0           57           M            PA                 2682     2749   
1           58           M            PA                 2894     2729   
2           58           M            PA                 2500     2048   
3           80           M            PA                 2500     2048   
4           74           F            PA                 2500     2048   

   OriginalImagePixelSpacing[x     y]  
0                        0.143  0.143  
1                        0.143  0.143  
2 

In [2]:
# merge test and valid cuz i didnt download all of the dataset
test_df = pd.read_csv("../data/csv/nih-cxr-lt_single-label_test.csv")
train_df = pd.read_csv("../data/csv/nih-cxr-lt_single-label_train.csv")
print(test_df.shape)
print(train_df.shape)
frames = [test_df,train_df]

merged_df = pd.concat(frames).drop_duplicates()
print(merged_df.shape)


(20279, 22)
(68058, 22)
(88337, 22)


In [25]:
merged_df.to_csv("../data/csv/merged_dataset.csv", index=False)

In [ ]:
# # get all 
# heart_df = df[df["Finding Labels"].str.contains("Atelectasis|Cardiomegaly|Edema|Effusion|Tortuous Aorta|Calcification of the Aorta|No Finding ")]
# # df.size
# print("Size of the heart_df:",heart_df.size)

In [ ]:
# keep heart related stuff
heart_related_df = merged_df[
    merged_df[["Atelectasis", "Cardiomegaly", "Edema", "Effusion", 
               "Tortuous Aorta", "Calcification of the Aorta"]].any(axis=1)
]
no_finding_df = merged_df[
    (merged_df.drop(columns=["id", "subject_id", "No Finding"]) == 0).all(axis=1) &
    (merged_df["No Finding"] == 1)
]
no_finding_sample = no_finding_df.sample(frac=0.067, random_state=42)

balanced_df = pd.concat([heart_related_df, no_finding_sample], ignore_index=True)

columns_to_drop = ["Consolidation", "Emphysema", "Fibrosis", "Hernia", "Infiltration",
                   "Mass", "Nodule", "Pleural_Thickening", "Pneumonia", "Pneumoperitoneum",
                   "Pneumothorax", "Pneumomediastinum", "Subcutaneous Emphysema"]
balanced_df = balanced_df.drop(columns=columns_to_drop, errors="ignore")

balanced_df.to_csv("../data/csv/filtered_dataset.csv", index=False)

print(balanced_df.shape)

(13990, 9)


In [ ]:
# sort by the csv
df = pd.read_csv("../data/csv/filtered_dataset.csv")

df = df.sort_values(by='id',key=lambda col: col.str.extract(r'(\d+)_(\d+)').astype(int).apply(tuple, axis=1))

# Save it
df.to_csv("../data/csv/filtered_dataset.csv", index=False)


In [ ]:
# traverse the images and compares if it exists then adds it into a new csv
import shutil
filtered_df = pd.read_csv("../data/csv/filtered_dataset.csv")
image_id = set(filtered_df["id"].tolist())

print(image_id)

root_dir = "../data/"

# destination
filtered_images_dir = "../data/filtered_images/"
os.makedirs(filtered_images_dir,exist_ok=True)

for folder_name in os.listdir(root_dir):
    folder_path = os.path.join(root_dir,folder_name)
    # print("folder path: ", folder_path)
    if os.path.isdir(folder_path) and folder_name.startswith("images_"):
        images_subfolder = os.path.join(folder_path,"images")
        if os.path.exists(images_subfolder):
            for img_filename in os.listdir(images_subfolder):
                if img_filename in image_id:
                    src_path = os.path.join(images_subfolder,img_filename)
                    dst_path = os.path.join(filtered_images_dir,img_filename)
                    shutil.copy(src_path,dst_path)
                
print("finidhsed")

In [27]:
# splitting the dataset

from sklearn.model_selection import train_test_split

merged_df = pd.read_csv("../data/csv/filtered_dataset.csv")
calcification_aorta_df = merged_df[merged_df["Calcification of the Aorta"] == 1]

tortuous_aorta_df =  merged_df[merged_df["Tortuous Aorta"] == 1]

edema_df =  merged_df[merged_df["Edema"] == 1]

cardiomegaly_df =  merged_df[merged_df["Cardiomegaly"] == 1]


print("Calcification: ",calcification_aorta_df.shape)
print("Tortuous: ",tortuous_aorta_df.shape)
print("Edema: ",edema_df.shape)
print("Cardiomegaly: ",cardiomegaly_df.shape)

# CALCIFICATION OF AORTA
# 90% training 
train_calc, temp_calc = train_test_split(
    calcification_aorta_df,
    test_size=0.10,
    random_state=50,
    shuffle = True
)
# 5% testing 5% validation
val_calc, test_calc = train_test_split(
    temp_calc,
    test_size=0.5,
    random_state=50,
    shuffle=True
)
# TORTUOUS AORTA
# 90% training 
train_tort, temp_tort = train_test_split(
    tortuous_aorta_df,
    test_size=0.10,
    random_state=50,
    shuffle = True
)
# 5% testing 5% validation
val_tort, test_tort = train_test_split(
    temp_tort,
    test_size=0.5,
    random_state=50,
    shuffle=True
)

# EDEMA
# 90% training 
train_edema, temp_edema = train_test_split(
    edema_df,
    test_size=0.10,
    random_state=50,
    shuffle = True
)
# 5% testing 5% validation
val_edema, test_edema = train_test_split(
    temp_edema,
    test_size=0.5,
    random_state=50,
    shuffle=True
)

# CARDIOMEGALY
# 90% training 
train_card, temp_card = train_test_split(
    cardiomegaly_df,
    test_size=0.10,
    random_state=50,
    shuffle = True
)
# 5% testing 5% validation
val_card, test_card = train_test_split(
    temp_card,
    test_size=0.5,
    random_state=50,
    shuffle=True
)


train_dataset_df = pd.concat([train_edema,train_calc,train_card,train_tort], ignore_index=True)
validation_dataset_df = pd.concat([val_edema,val_calc,val_card,val_tort], ignore_index=True)
test_dataset_df = pd.concat([test_edema,test_calc,test_card,test_tort], ignore_index=True)

# remove duplicates bisag wala jud ba 
train_dataset_df = train_dataset_df.drop_duplicates(subset=["id"]).reset_index(drop=True)
validation_dataset_df = validation_dataset_df.drop_duplicates(subset=["id"]).reset_index(drop=True)
test_dataset_df = test_dataset_df.drop_duplicates(subset=["id"]).reset_index(drop=True)

print(train_dataset_df.shape)
print(validation_dataset_df.shape)
print(test_dataset_df.shape)

print(train_calc.head(2))







Calcification:  (185, 9)
Tortuous:  (446, 9)
Edema:  (599, 9)
Cardiomegaly:  (1034, 9)
(2036, 9)
(113, 9)
(115, 9)
                     id  Atelectasis  Cardiomegaly  Edema  Effusion  \
10030  00019176_047.png            0             0      0         0   
9209   00017555_005.png            0             0      0         0   

       Tortuous Aorta  Calcification of the Aorta  No Finding  subject_id  
10030               0                           1           0       19176  
9209                0                           1           0       17555  


In [ ]:
# splits the common ones and merging it with the one on top
common_df = merged_df[(merged_df["No Finding"] == 1) | (merged_df["Effusion"] == 1) |(merged_df["Atelectasis"] == 1)]

train_common, temp_common = train_test_split(
    common_df,
    test_size=0.10,  
    random_state=50,
    shuffle=True,
    stratify=common_df[["No Finding", "Effusion", "Atelectasis"]]  
)

val_common, test_common = train_test_split(
    temp_common,
    test_size=0.5,
    random_state=50,
    shuffle=True,
    stratify=temp_common[["No Finding", "Effusion", "Atelectasis"]]
)

new_train_dataset_df = pd.concat([train_dataset_df, train_common], ignore_index=True)
new_validation_dataset_df = pd.concat([validation_dataset_df, val_common], ignore_index=True)
new_test_dataset_df = pd.concat([test_dataset_df, test_common], ignore_index=True)

new_train_dataset_df = new_train_dataset_df.drop_duplicates(subset=["id"]).reset_index(drop=True)
new_validation_dataset_df = new_validation_dataset_df.drop_duplicates(subset=["id"]).reset_index(drop=True)
new_test_dataset_df = new_test_dataset_df.drop_duplicates(subset=["id"]).reset_index(drop=True)

print(new_train_dataset_df.shape)
print(new_validation_dataset_df.shape)
print(new_test_dataset_df.shape)

no_finding_count = (new_train_dataset_df["Tortuous Aorta"] == 1).sum()
print(f'Number of "Tortuous Aorta" cases in train dataset: {no_finding_count}')

no_finding_count = (new_test_dataset_df["Tortuous Aorta"] == 1).sum()
print(f'Number of "No Finding" cases in train dataset: {no_finding_count}')

no_finding_count = (new_validation_dataset_df["Tortuous Aorta"] == 1).sum()
print(f'Number of "No Finding" cases in train dataset: {no_finding_count}')

no_finding_count = (new_train_dataset_df["Calcification of the Aorta"] == 1).sum()
print(f'Number of "No Finding" cases in train dataset: {no_finding_count}')

no_finding_count = (new_test_dataset_df["Calcification of the Aorta"] == 1).sum()
print(f'Number of "No Finding" cases in train dataset: {no_finding_count}')

no_finding_count = (new_validation_dataset_df["Calcification of the Aorta"] == 1).sum()
print(f'Number of "No Finding" cases in train dataset: {no_finding_count}')

no_finding_count = (new_train_dataset_df["No Finding"] == 1).sum()
print(f'Number of "No Finding" cases in train dataset: {no_finding_count}')

no_finding_count = (new_test_dataset_df["No Finding"] == 1).sum()
print(f'Number of "No Finding" cases in train dataset: {no_finding_count}')

no_finding_count = (new_validation_dataset_df["No Finding"] == 1).sum()
print(f'Number of "No Finding" cases in train dataset: {no_finding_count}')




(12589, 9)
(699, 9)
(702, 9)
Number of "No Finding" cases in train dataset: 401
Number of "No Finding" cases in train dataset: 23
Number of "No Finding" cases in train dataset: 22
Number of "No Finding" cases in train dataset: 166
Number of "No Finding" cases in train dataset: 10
Number of "No Finding" cases in train dataset: 9
Number of "No Finding" cases in train dataset: 3527
Number of "No Finding" cases in train dataset: 196
Number of "No Finding" cases in train dataset: 196


In [40]:
# save it inito csv

new_train_dataset_df = new_train_dataset_df.sort_values(by='id',key=lambda col: col.str.extract(r'(\d+)_(\d+)').astype(int).apply(tuple, axis=1))
new_test_dataset_df = new_test_dataset_df.sort_values(by='id',key=lambda col: col.str.extract(r'(\d+)_(\d+)').astype(int).apply(tuple, axis=1))
new_validation_dataset_df = new_validation_dataset_df.sort_values(by='id',key=lambda col: col.str.extract(r'(\d+)_(\d+)').astype(int).apply(tuple, axis=1))



new_train_dataset_df.to_csv("../data/csv/train_data.csv", index=False)
new_test_dataset_df.to_csv("../data/csv/test_data.csv", index=False)
new_validation_dataset_df.to_csv("../data/csv/valid_data.csv", index=False)


In [41]:
# traverse through the training_data csv and save all images into train_images folder

train_df = pd.read_csv("../data/csv/train_data.csv")
image_id = set(train_df["id"].tolist())

print(image_id)

root_dir = "../data/"
# sauce
source_dir = "../data/filtered_images/"
# destination
train_dir = "../data/train_images/"
os.makedirs(train_dir,exist_ok=True)

for img_filename in os.listdir(source_dir):
    if img_filename in image_id:
        src_path = os.path.join(source_dir,img_filename)
        dst_path = os.path.join(train_dir,img_filename)
        shutil.copy(src_path,dst_path)

print("yay")
                

{'00005632_001.png', '00008518_001.png', '00003421_001.png', '00000544_000.png', '00011973_021.png', '00000450_002.png', '00006142_012.png', '00005200_001.png', '00011498_000.png', '00022993_021.png', '00023111_008.png', '00024460_000.png', '00026362_000.png', '00015516_000.png', '00017792_003.png', '00003737_000.png', '00020671_018.png', '00025769_019.png', '00001320_003.png', '00007352_012.png', '00010096_000.png', '00010315_005.png', '00001330_002.png', '00004189_000.png', '00017403_011.png', '00021420_015.png', '00026161_003.png', '00005014_014.png', '00003957_002.png', '00014034_017.png', '00015843_009.png', '00005403_011.png', '00016776_000.png', '00019010_001.png', '00003395_007.png', '00007667_001.png', '00022687_000.png', '00005681_053.png', '00004344_040.png', '00006297_004.png', '00015383_000.png', '00025189_011.png', '00009665_005.png', '00011402_019.png', '00015262_007.png', '00008008_036.png', '00006134_002.png', '00005778_017.png', '00025768_007.png', '00028358_002.png',

In [42]:
# traverse through the test_data csv and save all images into test_images folder

train_df = pd.read_csv("../data/csv/test_data.csv")
image_id = set(train_df["id"].tolist())

print(image_id)

root_dir = "../data/"
# sauce
source_dir = "../data/filtered_images/"
# destination
train_dir = "../data/test_images/"
os.makedirs(train_dir,exist_ok=True)

for img_filename in os.listdir(source_dir):
    if img_filename in image_id:
        src_path = os.path.join(source_dir,img_filename)
        dst_path = os.path.join(train_dir,img_filename)
        shutil.copy(src_path,dst_path)

print("yay")
                

{'00010571_001.png', '00009356_004.png', '00030519_001.png', '00026072_019.png', '00019639_011.png', '00000955_002.png', '00014236_008.png', '00011380_005.png', '00013752_001.png', '00029532_025.png', '00009845_040.png', '00018918_003.png', '00020398_004.png', '00005537_012.png', '00016009_046.png', '00014795_002.png', '00027961_003.png', '00017643_000.png', '00013993_065.png', '00022782_002.png', '00003628_001.png', '00026346_006.png', '00002742_000.png', '00027700_001.png', '00016987_005.png', '00018921_054.png', '00004911_014.png', '00006057_003.png', '00030300_002.png', '00010585_027.png', '00029357_006.png', '00010638_001.png', '00013176_003.png', '00006054_000.png', '00023325_033.png', '00003548_004.png', '00009602_001.png', '00015170_006.png', '00021484_000.png', '00025124_004.png', '00027923_000.png', '00012904_004.png', '00006296_009.png', '00016896_006.png', '00019216_001.png', '00007576_028.png', '00016868_000.png', '00017233_002.png', '00003382_007.png', '00014728_000.png',

In [43]:
# traverse through the val_data csv and save all images into val_images folder

train_df = pd.read_csv("../data/csv/valid_data.csv")
image_id = set(train_df["id"].tolist())

print(image_id)

root_dir = "../data/"
# sauce
source_dir = "../data/filtered_images/"
# destination
train_dir = "../data/validation_images/"
os.makedirs(train_dir,exist_ok=True)

for img_filename in os.listdir(source_dir):
    if img_filename in image_id:
        src_path = os.path.join(source_dir,img_filename)
        dst_path = os.path.join(train_dir,img_filename)
        shutil.copy(src_path,dst_path)

print("yay")
                

{'00030571_002.png', '00003504_001.png', '00007018_058.png', '00028897_015.png', '00015691_000.png', '00003110_000.png', '00022454_006.png', '00026801_005.png', '00007551_001.png', '00011579_035.png', '00022607_009.png', '00025924_018.png', '00014414_010.png', '00000798_036.png', '00013485_001.png', '00013520_022.png', '00018360_014.png', '00004759_000.png', '00007679_001.png', '00000057_000.png', '00018366_031.png', '00022764_001.png', '00009841_000.png', '00020716_002.png', '00027706_033.png', '00004299_005.png', '00005003_000.png', '00028354_001.png', '00018180_001.png', '00006634_000.png', '00012060_004.png', '00007520_003.png', '00020113_009.png', '00010535_012.png', '00021107_002.png', '00000954_006.png', '00012157_000.png', '00005545_000.png', '00003078_000.png', '00023325_005.png', '00000814_001.png', '00002412_008.png', '00010547_003.png', '00015484_000.png', '00018379_009.png', '00025382_002.png', '00027379_000.png', '00000591_019.png', '00000877_029.png', '00002219_003.png',

In [53]:
# ==============================================================================
# drop subject id for each of the csvs

# train = pd.read_csv("../data/csv/train_data.csv")
# train = train.drop(columns=["subject_id"])
# # train.head(1)
# train.to_csv("../data/csv/train_data.csv")

# test = pd.read_csv("../data/csv/test_data.csv")
# test = test.drop(columns=["subject_id"])
# # train.head(1)
# test.to_csv("../data/csv/test_data.csv")

val = pd.read_csv("../data/csv/valid_data.csv")
val = val.drop(columns=["subject_id"])
# train.head(1)
val.to_csv("../data/csv/valid_data.csv")

In [44]:
# ==============================================================================

In [1]:
# reads an image and turns it into an array
import cv2

def load_image(image_path,size = (224,224)):
    

    img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,size)
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img,axis = -1)
    return img

In [2]:
# save images into numpy (training)

LABEL_COLUMNS = ["Atelectasis","Cardiomegaly","Edema","Effusion","Tortuous Aorta","Calcification of the Aorta","No Finding"]

def create_dataset(csv_path,image_folder):
    df = pd.read_csv(csv_path)
    X = []
    Y = []

    for i, row in df.iterrows():
        img_filename = row['id']
        img_path = os.path.join(image_folder,img_filename)

        if os.path.exists(img_path):
            img = load_image(img_path)
            X.append(img)
            labels = row[LABEL_COLUMNS].values.astype(np.int8)
            Y.append(labels)
            # print(f"Loaded: {img_path}")  
        # else:
            # print(f"Image not found: {img_path}")
    X = np.stack(X)
    Y = np.stack(Y)
    return X,Y





In [5]:
# turns every image to numpy test

test_csv_path = "../data/csv/test_data.csv"
test_image_path = "../data/test_images/"

X_test, Y_test = create_dataset(test_csv_path,test_image_path)

X_test = X_test.astype(dtype=np.float32)
Y_test = Y_test.astype(dtype=np.float32)

np.save("../data/numpy/X_test.npy", X_test)
np.save("../data/numpy/Y_test.npy", Y_test)


In [6]:
# turns every image to numpy train

train_csv_path = "../data/csv/train_data.csv"
train_image_path = "../data/train_images/"

X_train, Y_train = create_dataset(train_csv_path,train_image_path)

X_train = X_train.astype(dtype=np.float32)
Y_train = Y_train.astype(dtype=np.float32)

np.save("../data/numpy/X_train.npy", X_train)
np.save("../data/numpy/Y_train.npy", Y_train)


In [ ]:
# turns every image to numpy train

valid_csv_path = "../data/csv/valid_data.csv"
valid_image_path = "../data/validation_images/"

X_valid, Y_valid = create_dataset(valid_csv_path,valid_image_path)

X_valid = X_valid.astype(dtype=np.float32)
Y_valid = Y_valid.astype(dtype=np.float32)

np.save("../data/numpy/X_valid.npy", X_valid)
np.save("../data/numpy/Y_valid.npy", Y_valid)


In [2]:
X_train = np.load("../data/numpy/X_train.npy")
Y_train = np.load("../data/numpy/Y_train.npy")

X_train = X_train.astype(dtype=np.float32)
Y_train = Y_train.astype(dtype=np.float32)

np.save("../data/numpy/X_train.npy", X_train)
np.save("../data/numpy/Y_train.npy", Y_train)


In [3]:
X_valid = np.load("../data/numpy/X_valid.npy")
Y_valid = np.load("../data/numpy/Y_valid.npy")


X_valid = X_valid.astype(dtype=np.float32)
Y_valid = Y_valid.astype(dtype=np.float32)


np.save("../data/numpy/X_valid.npy", X_valid)
np.save("../data/numpy/Y_valid.npy", Y_valid)